In [ ]:
# either by directly passing api_key to the client or by setting the OPENAI_API_KEY environment variable
# .env file content
OPENAI_API_KEY = ""
ANTHROPIC_API_KEY = ""
GOOGLE_API_KEY = ""
HUGGINGFACEHUB_ACCESS_TOKEN = ""

In [ ]:
import dotenv
dotenv.load_dotenv()
import os
# print( os.getenv('GOOGLE_API_KEY') )

## Direct client call
```%pip install openai anthropic google-genai```

[API Pricing Compare](https://yourgpt.ai/tools/openai-and-other-llm-api-pricing-calculator)

[OpenAI Models List](https://platform.openai.com/docs/models#:~:text=Cost%2Doptimized%20models)

[Anthropic Models List](https://docs.anthropic.com/en/docs/about-claude/models/overview#model-aliases)

[Google Models List](https://ai.google.dev/gemini-api/docs/models#model-variations)

In [ ]:
from openai import OpenAI
client = OpenAI()
completion = client.chat.completions.create(model='gpt-4.1-nano', messages=[{"role": "user", "content": "Hello world"}])
print(completion.choices[0].message.content)

In [ ]:
import anthropic
client = anthropic.Anthropic()
message = client.messages.create(model='claude-3-haiku-20240307', messages=[{"role": "user", "content": "Hello world"}], max_tokens=100)
print(message.content[0].text)

In [ ]:
from google import genai
client = genai.Client()
response = client.models.generate_content(model="gemini-2.0-flash", contents="Hello world")
print(response.text)

## Using langchain unified interface for Models
```%pip install langchain_openai langchain_anthropic langchain-google-genai```

[OpenAI API Ref](https://python.langchain.com/api_reference/openai/index.html) / [OpenAI Integration Docs](https://python.langchain.com/docs/integrations/providers/openai/)

[Anthropic API Ref](https://python.langchain.com/api_reference/anthropic/index.html) / [Anthropic Integration Docs](https://python.langchain.com/docs/integrations/providers/anthropic/)

[Google API Ref](https://python.langchain.com/api_reference/google_genai/index.html#module-langchain_google_genai) / [Google Integration Docs](https://python.langchain.com/docs/integrations/providers/google/)


In [ ]:
from langchain_openai import ChatOpenAI # they are chat models (not LLM or embedding models)
model = ChatOpenAI(model='gpt-4.1-nano')# optional temperature 0~2 -> measure of randomness; top_p -> diverse word collection
                                        # max_completion_tokens -> op token size, restrict for lower api usage cost
result = model.invoke('Hello world')
print(result)
print(result.content)                   # answer only

In [ ]:
from langchain_anthropic import ChatAnthropic
model = ChatAnthropic(model='claude-3-haiku-20240307')
result = model.invoke('Hello world')
print(result.content)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash')
result = model.invoke('Hello world')
print(result.content)

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
llm = HuggingFaceEndpoint(repo_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0", task="text-generation") # EXTRA: model_kwargs={"temperature": 0.3, "max_length": 50}
model = ChatHuggingFace(llm=llm)
result = model.invoke("Who is first PM of India? Answer in two words.")
print(result.content)

In [ ]:
import os
os.environ['HF_HOME'] = 'D:/huggingface/models' # optional

# for hf in local - Pipeline
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
llm=HuggingFacePipeline.from_model_id(  model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0", task="text-generation",
                                        pipeline_kwargs={"temperature": 0.3, "max_length": 50}  )
model = ChatHuggingFace(llm=llm)
result = model.invoke("What are the 3 parts of CPU?")
print(result.content)

## Embedding Models

In [ ]:
# OpenAI Embeddings - for single query
from langchain_openai import OpenAIEmbeddings

In [ ]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=32) # vector dim of 32 items, greater captures more context
result = embedding.embed_query("3 parts of CPU are ALU, CU, MU.")
print(str(result))

In [ ]:
# OpenAI Embeddings - for docs/texts (multiple query)
embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=128)
texts = ["Who are you?", "Whats the full form of WHO?", "What is full moon"]
result = embedding.embed_documents(texts)
print(str(result))   # size (3,128)

In [ ]:
# HF local embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
text = "This is a test document."
result = embedding.embed_query(text) # or .embed_documents(texts)
print(str(result))

## Document Similarity - Nano Project

In [ ]:
from langchain_openai import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large', dimension=300)

In [ ]:
documents = ["...","...","...","...","..."] # like related to indian cricket team members
query = "Tell me about Virat Kohli"

In [ ]:
doc_embeddings = embedding.embed_documents(documents) # 5 vectors, each have 300 dim
query_embedding = embedding.embed_query(query)

In [ ]:
cosine_similarities = cosine_similarity([query_embedding], doc_embeddings) # both input should be 2D list, output also 2D
similarity = cosine_similarities[0] # [ , , , , ]

In [ ]:
index, score = sorted(  list(enumerate(similarity)) , key=lambda x:x[1]  )[-1]
index, score

In [ ]:
print(documents[index]) # the highest likely document
print(score) # respective similarity score